In [13]:
from pyspark.sql.functions import col, split, concat_ws, when, size

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("TrackTimePrediction") \
    .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/12 00:06:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
file_path1 = "./tfrrs_dataall.csv"
file_path2 = "./tfrrs_remaining_archives.csv"
file_path3 = "./tfrrs_current.csv"

df1 = spark.read.csv(file_path1, header=True, inferSchema=True)
df2 = spark.read.csv(file_path2, header=True, inferSchema=True)
df3 = spark.read.csv(file_path3, header=True, inferSchema=True)

combined_df = df1.unionByName(df2).unionByName(df3)

combined_df.printSchema()

combined_df.show(10)

root
 |-- _c0: integer (nullable = true)
 |-- athlete_link: string (nullable = true)
 |-- athlete_name: string (nullable = true)
 |-- meet_date: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- team_link: string (nullable = true)
 |-- team_name: string (nullable = true)
 |-- time: string (nullable = true)
 |-- year: string (nullable = true)

+---+--------------------+----------------+------------+----+--------------------+--------------+--------+----+
|_c0|        athlete_link|    athlete_name|   meet_date|rank|           team_link|     team_name|    time|year|
+---+--------------------+----------------+------------+----+--------------------+--------------+--------+----+
|  0|https://tf.tfrrs....|   Wolfe, Parker| May 9, 2024|   1|https://tf.tfrrs....|North Carolina|13:19.38|JR-3|
|  1|https://tf.tfrrs....|Mullarkey, David|Apr 26, 2024|   2|https://tf.tfrrs....| Florida State|13:28.43|JR-3|
|  2|https://tf.tfrrs....|    Martin, Gary|Mar 28, 2024|   3|https://tf.tfrrs.

25/06/11 14:34:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
 Schema: _c0, athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
Expected: _c0 but found: 
CSV file: file:///Users/emccauley6/Documents/GitHub/prism/tfrrs_dataall.csv


In [16]:
output_path = "./tfrrs_database"
combined_df.coalesce(1).write \
    .option("header", "true") \
    .mode("append") \
    .csv(output_path)

25/06/11 14:35:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
 Schema: _c0, athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
Expected: _c0 but found: 
CSV file: file:///Users/emccauley6/Documents/GitHub/prism/tfrrs_dataall.csv
25/06/11 14:35:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
 Schema: _c0, athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
Expected: _c0 but found: 
CSV file: file:///Users/emccauley6/Documents/GitHub/prism/tfrrs_remaining_archives.csv
25/06/11 14:35:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , athlete_link, athlete_name, meet_date, rank, team_link, team_name, time, year
 Schema: _c0, athlete_link, athlete_name, meet_date, rank, team_link, team_name, t

In [39]:
spark.stop()

ConnectionRefusedError: [Errno 61] Connection refused

In [8]:
file_path1 = "./tfrrs_database/tfrrs_raw.csv"
df = spark.read.csv(file_path1, header=True, inferSchema=True)
print(df.count())

90148


In [12]:
print(df.dropDuplicates(['athlete_name', 'time', 'meet_date', 'year', 'team_name']).count())


68568


In [ ]:
df_no_dupes = df.dropDuplicates(['athlete_name', 'time', 'meet_date', 'year', 'team_name'])
df_no_dupes.show(10)


+-----+--------------------+--------------------+------------+----+--------------------+------------------+--------+----+
|  _c0|        athlete_link|        athlete_name|   meet_date|rank|           team_link|         team_name|    time|year|
+-----+--------------------+--------------------+------------+----+--------------------+------------------+--------+----+
|78526|https://tf.tfrrs....|        Aardal, Erik| Mar 4, 2010|  24|https://tf.tfrrs....|  UC Santa Barbara|14:57.19|SR-4|
|18044|https://tf.tfrrs....|     Abberton , Rory|Apr 13, 2022|   4|https://tf.tfrrs....|       Chico State|14:26.48|SR-4|
|17520|https://tf.tfrrs....|         Abbes, Gabe|May 13, 2022|  58|https://tf.tfrrs....|        California|14:55.80|FR-1|
|13378|https://tf.tfrrs....|       Abbott, Devin|Apr 14, 2023| 200|https://tf.tfrrs....|St. Joseph's (Me.)|16:44.19|SR-4|
|24503|https://tf.tfrrs....|Abdellatif, Shimeles|Apr 24, 2021|   7|https://tf.tfrrs....|      Prairie View|15:56.58|FR-1|
|  698|https://tf.tfrrs.

In [ ]:
output_path = "./tfrrs_database"
df_no_dupes.coalesce(1).write \
    .option("header", "true") \
    .mode("append") \
    .csv(output_path)

Clean the milesplit dataset to only have names not names and schools

In [21]:
file_path1 = "./basic_HS_database.csv"
milesplit_df = spark.read.csv(file_path1, header=True, inferSchema=True)
milesplit_df.show(10)

+---+----------+--------------------+-----+-----+------+-----+-----+-----+------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+
|_c0|Unnamed: 0|      Athlete/School|Grade|State|Gender|FR_5k|SO_5k|JR_5k| SR_5k|FR_3200|SO_3200|JR_3200|SR_3200|FR_1600|SO_1600|JR_1600|SR_1600|FR_800|SO_800|JR_800|SR_800|
+---+----------+--------------------+-----+-----+------+-----+-----+-----+------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+
|  0|         0|A Bremer Atascade...| 2009|   CA|  Boys| NULL| NULL| NULL|  NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|  NULL|  NULL|  NULL| 125.3|
|  1|         1|A Lopez Paso Robl...| 2010|   CA|  Boys| NULL| NULL| NULL|  NULL|   NULL|  633.6|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|  NULL|  NULL|  NULL|  NULL|
|  2|         2|A j Stevens Sulta...| 2012|   CA|  Boys| NULL| NULL| NULL| 982.7|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL| 

25/06/11 15:24:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, Athlete/School, Grade, State, Gender, FR_5k, SO_5k, JR_5k, SR_5k, FR_3200, SO_3200, JR_3200, SR_3200, FR_1600, SO_1600, JR_1600, SR_1600, FR_800, SO_800, JR_800, SR_800
 Schema: _c0, Unnamed: 0, Athlete/School, Grade, State, Gender, FR_5k, SO_5k, JR_5k, SR_5k, FR_3200, SO_3200, JR_3200, SR_3200, FR_1600, SO_1600, JR_1600, SR_1600, FR_800, SO_800, JR_800, SR_800
Expected: _c0 but found: 
CSV file: file:///Users/emccauley6/Documents/GitHub/prism/basic_HS_database.csv


In [26]:
milesplit_just_names = milesplit_df.withColumn(
    "athlete_name",
    concat_ws(
        " ", # The separator you want to use
        split(col("Athlete/School"), " ").getItem(0), # The first word
        split(col("Athlete/School"), " ").getItem(1)  # The second word
    )
)
milesplit_just_names.select("Athlete/School", "athlete_name").show(truncate=False)

+--------------------------------------------+-----------------+
|Athlete/School                              |athlete_name     |
+--------------------------------------------+-----------------+
|A Bremer Atascadero (CS)                    |A Bremer         |
|A Lopez Paso Robles (CS)                    |A Lopez          |
|A j Stevens Sultana (SS)                    |A j              |
|A. J Kimura El Toro (SS)                    |A. J             |
|A. J Sommers Elk Grove (SJ)                 |A. J             |
|A. J. Aldrian Murrieta Valley (SS)          |A. J.            |
|A.j Licon Alta Loma (SS)                    |A.j Licon        |
|A.j Singh Wood (Will C.) (SJ)               |A.j Singh        |
|A.j. Sager El Capitan (SD)                  |A.j. Sager       |
|AK Metoyer Claremont (SS)                   |AK Metoyer       |
|ANDREW PIERCE North Monterey County (CC)    |ANDREW PIERCE    |
|ANTHONY GOTTI St. Francis Salesian Prep (CC)|ANTHONY GOTTI    |
|Aakarsh Gopisetty Washin

In [27]:
output_path = "./milesplit_database"
milesplit_just_names.coalesce(1).write \
    .option("header", "true") \
    .mode("overwrite") \
    .csv(output_path)

25/06/11 15:32:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, Athlete/School, Grade, State, Gender, FR_5k, SO_5k, JR_5k, SR_5k, FR_3200, SO_3200, JR_3200, SR_3200, FR_1600, SO_1600, JR_1600, SR_1600, FR_800, SO_800, JR_800, SR_800
 Schema: _c0, Unnamed: 0, Athlete/School, Grade, State, Gender, FR_5k, SO_5k, JR_5k, SR_5k, FR_3200, SO_3200, JR_3200, SR_3200, FR_1600, SO_1600, JR_1600, SR_1600, FR_800, SO_800, JR_800, SR_800
Expected: _c0 but found: 
CSV file: file:///Users/emccauley6/Documents/GitHub/prism/basic_HS_database.csv


Remove some results from the hs database to limit faulty merging between datasets. Someone with only 2 high school results likely did not run in college, but could have the same name as someone who did and cause a poor merge since I can only go on name.

In [30]:
file_path1 = "./milesplit_database/HS_athlete_names.csv"
milesplit_df = spark.read.csv(file_path1, header=True, inferSchema=True)
milesplit_df.show(10)

+---+----------+--------------------+-----+-----+------+-----+-----+-----+------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------------+
|_c0|Unnamed: 0|      Athlete/School|Grade|State|Gender|FR_5k|SO_5k|JR_5k| SR_5k|FR_3200|SO_3200|JR_3200|SR_3200|FR_1600|SO_1600|JR_1600|SR_1600|FR_800|SO_800|JR_800|SR_800|athlete_name|
+---+----------+--------------------+-----+-----+------+-----+-----+-----+------+-------+-------+-------+-------+-------+-------+-------+-------+------+------+------+------+------------+
|  0|         0|A Bremer Atascade...| 2009|   CA|  Boys| NULL| NULL| NULL|  NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|  NULL|  NULL|  NULL| 125.3|    A Bremer|
|  1|         1|A Lopez Paso Robl...| 2010|   CA|  Boys| NULL| NULL| NULL|  NULL|   NULL|  633.6|   NULL|   NULL|   NULL|   NULL|   NULL|   NULL|  NULL|  NULL|  NULL|  NULL|     A Lopez|
|  2|         2|A j Stevens Sulta...| 2012|   CA|  Boys| NULL| NU

In [35]:
milesplit_half_results = milesplit_df.na.drop(thresh=15)
output_path = "./milesplit_database"
milesplit_half_results.coalesce(1).write \
    .option("header", "true") \
    .mode("append") \
    .csv(output_path)

In [36]:
milesplit_five_thresh = milesplit_df.na.drop(thresh=12)
output_path = "./milesplit_database"
milesplit_five_thresh.coalesce(1).write \
    .option("header", "true") \
    .mode("append") \
    .csv(output_path)

25/06/11 20:27:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1185647 ms exceeds timeout 120000 ms
25/06/11 20:27:51 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/11 20:27:54 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

Cleaning college data set to only have PR's for each athlete instead of all results for each. Need to extract Ids first then do it by that

In [3]:
file_path1 = "./tfrrs_database/tfrrs_no_dupes.csv"
tfrrs_df = spark.read.csv(file_path1, header=True, inferSchema=True)
tfrrs_df.show(10)

+-----+--------------------+--------------------+------------+----+--------------------+------------------+--------+----+
|  _c0|        athlete_link|        athlete_name|   meet_date|rank|           team_link|         team_name|    time|year|
+-----+--------------------+--------------------+------------+----+--------------------+------------------+--------+----+
|78526|https://tf.tfrrs....|        Aardal, Erik| Mar 4, 2010|  24|https://tf.tfrrs....|  UC Santa Barbara|14:57.19|SR-4|
|18044|https://tf.tfrrs....|     Abberton , Rory|Apr 13, 2022|   4|https://tf.tfrrs....|       Chico State|14:26.48|SR-4|
|17520|https://tf.tfrrs....|         Abbes, Gabe|May 13, 2022|  58|https://tf.tfrrs....|        California|14:55.80|FR-1|
|13378|https://tf.tfrrs....|       Abbott, Devin|Apr 14, 2023| 200|https://tf.tfrrs....|St. Joseph's (Me.)|16:44.19|SR-4|
|24503|https://tf.tfrrs....|Abdellatif, Shimeles|Apr 24, 2021|   7|https://tf.tfrrs....|      Prairie View|15:56.58|FR-1|
|  698|https://tf.tfrrs.

In [15]:
tfrrs_id = tfrrs_df.withColumn('id', 
        when(
            size(split(col("athlete_link"), "/")) > 4,
            split(col("athlete_link"), "/").getItem(4)).otherwise(
                None
            ))
tfrrs_id.show(10)

+-----+--------------------+--------------------+------------+----+--------------------+------------------+--------+----+-------+
|  _c0|        athlete_link|        athlete_name|   meet_date|rank|           team_link|         team_name|    time|year|     id|
+-----+--------------------+--------------------+------------+----+--------------------+------------------+--------+----+-------+
|78526|https://tf.tfrrs....|        Aardal, Erik| Mar 4, 2010|  24|https://tf.tfrrs....|  UC Santa Barbara|14:57.19|SR-4| 906742|
|18044|https://tf.tfrrs....|     Abberton , Rory|Apr 13, 2022|   4|https://tf.tfrrs....|       Chico State|14:26.48|SR-4|6941226|
|17520|https://tf.tfrrs....|         Abbes, Gabe|May 13, 2022|  58|https://tf.tfrrs....|        California|14:55.80|FR-1|7732231|
|13378|https://tf.tfrrs....|       Abbott, Devin|Apr 14, 2023| 200|https://tf.tfrrs....|St. Joseph's (Me.)|16:44.19|SR-4|7392210|
|24503|https://tf.tfrrs....|Abdellatif, Shimeles|Apr 24, 2021|   7|https://tf.tfrrs....|  

In [16]:
output_path = "./tfrrs_database"

tfrrs_id.coalesce(1).write \
    .option("header", "true") \
    .mode("append") \
    .csv(output_path)

In [ ]:
bad_rows_df = tfrrs_id.where(size(split(col("athlete_link"), "/")) <= 4)
bad_rows_df.show(truncate=False)

+-----+------------+------------+------------+----+--------------------------------------------------------------------------+------------------+-----------+----+----+
|_c0  |athlete_link|athlete_name|meet_date   |rank|team_link                                                                 |team_name         |time       |year|id  |
+-----+------------+------------+------------+----+--------------------------------------------------------------------------+------------------+-----------+----+----+
|57503|#           |N/A         |Apr 10, 2015|18  |https://tf.tfrrs.org/teams/tf/NY_college_m_Stony_Brook.html               |Stony Brook       |4:25.48 (1)|N/A |NULL|
|57483|#           |N/A         |Apr 11, 2015|12  |https://tf.tfrrs.org/teams/tf/NY_college_m_Columbia.html                  |Columbia          |43.41      |N/A |NULL|
|57504|#           |N/A         |Apr 11, 2015|19  |https://tf.tfrrs.org/teams/tf/NY_college_m_Stony_Brook.html               |Stony Brook       |4:25.92 (1)|N/A